In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import scipy.stats as stats
from datetime import datetime

plt.rcParams['font.family'] = ['sans-serif']
plt.rcParams['font.sans-serif'] = ['SimHei'] # 用来正常显示中文标签`
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (8,5) #提前设置图片形状大小

%config InlineBackend.figure_format = 'svg' #在notebook中可以更好的显示，svg输出是一种向量化格式，缩放网页并不会导致图片失真。这行代码似乎只用在ipynb文件中才能使用。

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')  # 忽略一些warnings

# # This allows multiple outputs from a single jupyter notebook cell:
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
pd.set_option('expand_frame_repr', False)
pd.set_option('display.unicode.ambiguous_as_wide', True)
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 180)
pd.set_option('display.max.columns', None)

from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important; }</style>"))

from RemoteQuery import RemoteSrc
from utils import *
src = RemoteSrc()

import redis
import pickle

**基础知识**
- NoSQL (not only SQL)，非关系型数据库。在数据量极大的当下，频繁的查询导致传统的relational databases关系型数据库难以满足。Redis是非关系型数据库的一种，满足强一致性和高可用性。存储读取的速度极快，通常redis被称为数据结构服务器，因为值（value）可以是字符串(String)、哈希(Hash)、列表(list)、集合(sets)和有序集合(sorted sets)等类型。
- 非关系型数据库就是说表与表之间没有关联，数据之间无关系，容易拓展，同时具有极高的读写性能。
- redis是一个数据库，而且是一个**内存数据库**，所有数据都是储存在内存中。对于小型的个人电脑，如果内存有32G而且平常大多程序跑不满，那么这些空间使用来存储数据，比如整理好的股票/币的数据，那么极其方便，因为读写的速度会非常快。但是更多建议是在比如SGD的linux服务器上，服务器上1个T、5个T这种量级的内存，平常多个人程序并行一个月也不会跑满，这么多的RAM就可以使用redis来存储大批量的tick级整理好的数据。

**Terminal命令**

Linux服务器
- Account: yby@192.168.1.139 Password: yby&SGD!

windows和linux终端操作可能有不同，比如windows不需要使用"--daemonize yes"去启动守护进程
- 连接服务程序：redis-server --daemonize yes， 再连接客户端：redis-cli。一般默认端口是6379，host为local。如果后面加上raw，即redis-cli --raw，可以避免中文乱码
- vscode上可以操作terminal分屏，一边为redis的terminal另一边为linux

以下为linux指令：
- flushdb：清除当前数据库
- flushall: 清除整个redis的缓存
- keys *: 返回当前所有的keys
- type keyname: 返回key所储存值的类型
- info memory返回内存使用情况

**Python交互**

In [12]:
# NOTE:
# redis中默认key和value都是以bytes的形式储存的，keys()返回的结果也是bytes。而get()函数里面有encoding的操作所以函数既可以接受bytes也可以接受str的形式
# 如果我们希望查询某一个str类型的键是否在数据库中，需要使用encode(),将str转换为bytes，看是否在r.keys()中；使用decode将查询出的bytes值转换为str
# 打印出来的结果，如果前面带b'的字样就表明这个字符串是bytes。声明redis连接池中的decode_responses，可以对键值对进行默认编码，那么get()和keys()得到的都是str类型了

# 但是，需要注意pickle进行序列化的问题，使用pickle存储的时候，pickle会将对象dumps成bytes类型的字符串，并且该字符串无法使用utf-8进行decode，因此会导致在get时候会爆出异常
# 结论：redis只是为了存储一些str字符串等键值对，decode_responses=True；如果在redis使用中，会涉及对象，字典，列表等的存储,decode_responses=False。默认值为false



# 指明decode_responses=True获取str类型的键值对反馈
pool = redis.ConnectionPool(host='localhost', port=6379, decode_responses=True)
r = redis.Redis(connection_pool=pool)

# ===string类型
#   增加数据：set key value（如果key存在，则修改为新的value）
print(r.set('str_type', 'str_value'))  # 打印True
#   追加数据：append key value
print(r.append('str_type', '_new'))  # 打印13，字符长度
#   查看数据：get key
print(r.get('str_type'))

r.set('name', 'runoob')  # 设置 name 对应的值
print(r['name'])
print(r.get('name'))  # 取出键 name 对应的值
print(type(r.get('name')))

# ===list类型
#   在插入数据时，如果该键并不存在，Redis将为该键创建一个
#   在末尾添加数据（列表右边）
r.rpush('list_type', '2', 'xy', 'li_val_end')
#   在头部添加数据（列表左边）
r.lpush('list_type', '1', 'xy', 'li_val_start')

#   查看数据
#   数据为：['li_val_start', 'xy', '1', '2', 'xy', 'li_val_end']
#   下标范围：lrange key start stop
print(r.lrange('list_type', 0, 10))
#   指定下标：lindex key index
print(r.lindex('list_type', -1))

#   删除数据
#   从末尾删除（列表右边）：rpop key
print(r.rpop('list_type'))  # 打印删除的值
#   从头部删除（列表左边）：lpop key
print(r.lpop('list_type'))  # 打印删除的值
#   指定值删除：lrem key count(可以存在多个重复的值，指定value删除的次数) value
print(r.lrem('list_type', 2, 'xy'))  # 打印成功删除的个数

# ===hash类型
#   hash类型的值是一个键值对集合，如：h_test : { field1:value1, field2:value2,...}
#   添加数据：hset key field value
print(r.hset('hash_type', 'filed', 'value'))  # 打印成功添加数据的条数
#   查看域值：hget key field
print(r.hget('hash_type', 'filed'))
#   查看所有的field：hkeys key
print(r.hkeys('hash_type'))
#   查看所有的value：hvals key
print(r.hvals('hash_type'))
#   查看所有的键值对：hgetall key
print(r.hgetall('hash_type'))


r.hset("hash1", "k1", {"yes": 3})
r.hset("hash1", "k2", 2)
print(r.hkeys("hash1")) # 取hash中所有的key
print(r.hget("hash1", "k1"))    # 单个取hash的key对应的值
print(r.hmget("hash1", "k1", "k2")) # 多个取hash的key对应的值
r.hsetnx("hash1", "k2", "v3") # hsetnx在原key没有的时候才会新建
print(r.hget("hash1", "k2")) # 仍然结果为v2

# ===set类型
#   Set类型为无序的字符集合，元素具有唯一性， 不重复(自动去重)
#   添加数据：sadd key member1 [member2 ...]
print(r.sadd('set_type', 'va', 'vb', 'vc', 'vd'))  # 打印成功添加数据的条数
#   查看数据：smembers key
print(r.smembers('set_type'))
#   随机删除：spop key
print(r.spop('set_type'))  # 打印删除的值
#   指定删除：srem key member1 [member2 ...]
print(r.srem('set_type', 'va', 'vb'))  # 打印成功删除的个数

# ===zset类型
#   每一个成员都会有一个分数(score)与之关联
#   成员是唯一的，但是分数(score)却是可以重复的
#   比如把一个班级的学生分成几组

#   添加数据： zadd key score member [score2 member2 …]
#   打印成功添加数据的条数
print(r.zadd('zset_type',
                     1, 'val1', 1, 'val2', 1, 'val3',
                     4, 'val4', 4, 'val5',
                     8, 'val6'
                     ))

#   查看数据
#   根据索引：zrange key start stop
print(r.zrange('zset_type', 0, 3))
#   根据score：zrangebyscore key min max
#   查看 score 1 到 2 的值
print(r.zrangebyscore('zset_type', 1, 2))

#   删除数据
#   根据值：zrem key member [member …]
print(r.zrem('zset_type', 'val3', 'val4'))  # 打印成功删除的个数
#   根据索引：zremrangebyrank key start stop
print(r.zremrangebyrank('zset_type', 2, 4))  # 打印成功删除的个数
#   根据score：zremrangebyscore key min max
print(r.zremrangebyscore('zset_type', 1, 7))  # 打印成功删除的个数


# ===全局key操作
#   查看所有的key：keys *
print(r.keys())
#   查看key的类型：type key
print(r.type('set_type'))
#   exists key 查看key是否存在
print(r.exists('abcd'))  # 不存在返回False

#   改名：rename key new_key
#   如果不存在则报错：no such key
# print(r.rename('str_type', 'str_type_new'))
#   删除键值对：del key [key2 key3 ...]
print(r.delete('hash_type'))  # 打印成功删除的个数

#   设置过期时间：expire key seconds
print(r.expire('list_type', 60))  # 返回bool
#   persist key 删除过期时间
print(r.persist('list_type'))# 返回bool
#   ttl key 查看时间
#   -1：没设置过期时间      -2：不存在这个键
print(r.ttl('list_type'))


1

In [2]:
# 需要存储dataframe、orderdict等对象的时候，先使用pickle序列化然后存储，指明decode_responses=False
pool = redis.ConnectionPool(host='localhost', port=6379, decode_responses=False)
r = redis.Redis(connection_pool=pool)
df=pd.DataFrame([range(5)]*5,index=list('HELLO'),columns=list('HELLO'))
df_bytes = pickle.dumps(df)
r.hset("hash1", 'test_df', df_bytes)
df_bytes_from_redis = r.hget("hash1", "test_df")
df_from_redis = pickle.loads(df_bytes_from_redis)
print(df_from_redis)

In [8]:
a = r.hget("hash3", 'non')
a is None

True

In [6]:
df_bytes_from_redis = r.hget("tick_list", "20220429")
df_from_redis = pickle.loads(df_bytes_from_redis)
print(df_from_redis)

[server_time
2022-04-29 09:30:04    8.002430
2022-04-29 09:30:09    8.002825
2022-04-29 09:30:14    8.003162
2022-04-29 09:30:19    8.003340
2022-04-29 09:30:24    8.002895
                         ...   
2022-04-29 14:56:44    8.021880
2022-04-29 14:56:49    8.021969
2022-04-29 14:56:50    8.021969
2022-04-29 14:56:54    8.021952
2022-04-29 14:56:59    8.022202
Name: midquote, Length: 3081, dtype: float64, server_time
2022-04-29 09:30:00    2.930927
2022-04-29 09:30:03    2.927186
2022-04-29 09:30:06    2.931993
2022-04-29 09:30:09    2.932792
2022-04-29 09:30:12    2.930127
                         ...   
2022-04-29 14:56:45    2.962434
2022-04-29 14:56:48    2.962434
2022-04-29 14:56:51    2.962434
2022-04-29 14:56:54    2.962951
2022-04-29 14:56:57    2.963467
Name: midquote, Length: 4741, dtype: float64, server_time
2022-04-29 09:30:04    7.901218
2022-04-29 09:30:09    7.901640
2022-04-29 09:30:12    7.901640
2022-04-29 09:30:14    7.902114
2022-04-29 09:30:19    7.902214
       